In [208]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import os
import librosa as librosa
import warnings
warnings.filterwarnings('ignore')

In [209]:
path = os.getcwd()
path

'/home/aqeelali7/Documents/Galvanized/Capstone-2-Music-Genre-Classifier/data/hip-hop'

In [210]:
p = '/home/aqeelali7/Documents/Galvanized/Capstone-2-Music-Genre-Classifier/'

In [211]:
genres = ['hip-hop', 'classical', 'country', 'electronic', 'metal']

In [212]:
def extract_audio_features(y, sr):
        '''
        This function extracts audio features from an audio file.
                Parameters:
                        id (string): the audio track id 
                        y 
                        sr 
                Returns:
                        audio_features (DataFrame): the extracted audio features
        '''
        # Features to concatenate in the final dictionary
        features = {'chroma_sftf': None, 'rolloff': None, 'zero_crossing_rate': None, 'rmse': None,
                    'flux': None, 'contrast': None, 'flatness': None}

        # Count silence
        if 0 < len(y):
            y_sound, _ = librosa.effects.trim(y)
        features['sample_silence'] = len(y) - len(y_sound)            

        # Using librosa to calculate the features
        features['chroma_sftf'] = np.mean(
            librosa.feature.chroma_stft(y=y, sr=sr))
        features['rolloff'] = np.mean(
            librosa.feature.spectral_rolloff(y, sr=sr))
        features['zero_crossing_rate'] = np.mean(
            librosa.feature.zero_crossing_rate(y))
        features['rmse'] = np.mean(librosa.feature.rms(y))
        features['flux'] = np.mean(librosa.onset.onset_strength(y=y, sr=sr))
        features['contrast'] = np.mean(
            librosa.feature.spectral_contrast(y, sr=sr))
        features['flatness'] = np.mean(librosa.feature.spectral_flatness(y))

        # MFCC treatment
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
        for idx, v_mfcc in enumerate(mfcc):
            features['mfcc_{}'.format(idx)] = np.mean(v_mfcc)

        features['tempo'] = librosa.beat.tempo(y, sr=sr)[0]
        return features

In [ ]:

#     in each genre:

#     iterate to next genre



# add the list as the contents of the row of a dataframe

# create list of features for each song
chroma_sftf = []
flux = [] 
rolloff = []
zero_crossing_rate = []
contrast = []
flatness = []
rmse = []
tempo = []
genre = []
sr = 22050

# loop thru the list of genres
for i in range(len(genres)):

    data_dir = "data/"+genres[i]
    new_path = os.path.join(p,data_dir)
    os.chdir(new_path)
    
#   extract audio features using librosa library
#   store extracted audio features in a dataframe
    count = 0
    for track_num in range(len(os.listdir())):

        audio, amplitude = librosa.load((os.listdir()[track_num]))
        audio = np.array(audio)
        amplitude = np.array(amplitude)
        curr_song_feats = extract_audio_features(audio,sr)
        
        # for each song, append data to respective features list
        chroma_sftf.append(curr_song_feats['chroma_sftf'])
        flux.append(curr_song_feats['flux'])
        rolloff.append(curr_song_feats['rolloff'])
        zero_crossing_rate.append(curr_song_feats['zero_crossing_rate'])
        contrast.append(curr_song_feats['contrast'])
        flatness.append(curr_song_feats['flatness'])
        rmse.append(curr_song_feats['rmse'])
        tempo.append(curr_song_feats['tempo'])
        genre.append(genres[i])
        count += 1
        
        if count%100 == 0:
            print('100 songs done')
        
        
        
    print('done with ', genres[i])
    
        

        


In [220]:
col_feats = list(zip(genre,chroma_sftf, flux, rolloff, zero_crossing_rate, contrast,flatness,rmse,tempo))
col_names = ["genre", "chroma_sftf", "flux", "rolloff", "zero_crossing_rate", "contrast","flatness","rmse","tempo"]

df = pd.DataFrame(col_feats, columns = col_names)
df.head()

,genre,chroma_sftf,flux,rolloff,zero_crossing_rate,contrast,flatness,rmse,tempo
0,hip-hop,0.379869,1.436754,1289.819978,0.029480,21.493925,0.000617,0.377290,89.102909
1,hip-hop,0.486238,1.691776,5755.731863,0.131533,20.783028,0.053096,0.243562,92.285156
2,hip-hop,0.530458,1.888279,6261.773247,0.126315,19.989482,0.050299,0.324483,123.046875
3,hip-hop,0.485251,1.290328,3830.665725,0.062796,19.489805,0.013420,0.185147,129.199219
4,hip-hop,0.405980,1.543403,4310.546157,0.081577,22.124888,0.018615,0.266838,92.285156


In [221]:
plt.rc("font", size=14)
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
%matplotlib inline
import scipy.stats as scs
import scipy
from sklearn.linear_model import LogisticRegression, LinearRegression
import itertools
from sklearn.preprocessing import StandardScaler # data normalization
from sklearn.metrics import precision_score # evaluation metric
from sklearn.metrics import accuracy_score # evaluation metric
from sklearn.metrics import recall_score # evaluation metric
from sklearn.metrics import classification_report # evaluation metric
from sklearn.metrics import confusion_matrix # evaluation metric
from sklearn.metrics import log_loss # evaluation metric
from matplotlib import rcParams # plot size customization
rcParams['figure.figsize'] = (20, 10)

In [222]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 938 entries, 0 to 937
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   genre               938 non-null    object 
 1   chroma_sftf         938 non-null    float64
 2   flux                938 non-null    float64
 3   rolloff             938 non-null    float64
 4   zero_crossing_rate  938 non-null    float64
 5   contrast            938 non-null    float64
 6   flatness            938 non-null    float64
 7   rmse                938 non-null    float64
 8   tempo               938 non-null    float64
dtypes: float64(8), object(1)
memory usage: 66.1+ KB


In [223]:

y = df["genre"]
X = df.drop('genre',axis=1,inplace=False)


In [246]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)
lr = LogisticRegression(C = 1000, solver = 'liblinear')
lr.fit(X_train,y_train)
yhat = lr.predict(X_test)
yhat_prob = lr.predict_proba(X_test)

print("Accuracy Score : ", accuracy_score(yhat,y_test))

print("Precision Score : ",precision_score(y_test, yhat, 
                                           pos_label='positive',
                                           average='macro'))
print("Recall Score : ",recall_score(y_test, yhat, 
                                           pos_label='positive',
                                           average='macro'))

Accuracy Score :  0.5361702127659574
Precision Score :  0.5390789952675645
Recall Score :  0.5347188592951304


In [235]:
'''
gen = df.loc[:,"genre"]
dums = pd.get_dummies(gen,drop_first=True)
df.drop('genre',axis=1,inplace=True)
dums.head()
df = pd.concat([df,dums],axis=1)

df.head()
'''

'\ngen = df.loc[:,"genre"]\ndums = pd.get_dummies(gen,drop_first=True)\ndf.drop(\'genre\',axis=1,inplace=True)\ndums.head()\ndf = pd.concat([df,dums],axis=1)\n\ndf.head()\n'